# Consumer complaints topic extraction using Latent diriclect allocation (LDA) and Latent semantic analysis (LSA)
LSA is a method of analyzing text data using principal component analysis. LDA is a probabilistic approach, utilizing the Dirichlet distribution. Over-fitting is less likely with LDA since it uses priors.

Additionally, both documents and words are represented by orthogonal vectors. LDA is concerned with independent topics, while LSA is concerned with orthogonal representations.

Lastly, LDA requires a minimum number of topics or components. K is selected as the most effective method for capturing the majority of variations in the data.

# Comcast consumer complaint dataset

This dataset was obtained from kaggle and made readily available to all by charlie.H. This data set contains 2 CSV files.
1. comcast_consumeraffairs_complaints.csv
2. comcast_fcc_complaints_2015.csv

The first data set however is more beneficial for sentiment analysis. We will be working with the second dataset today comcast_fcc_complaints_2015.csv. it contains several columns including one with verbatim which we will use for topic extraction.

# Initializing packages and exploring the dataset

In [23]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')



df = pd.read_csv('comcast_fcc_complaints_2015.csv')
display(df)

[nltk_data] Downloading package wordnet to C:\Users\Kumarappan
[nltk_data]     M\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Kumarappan
[nltk_data]     M\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,Ticket #,Customer Complaint,Date,Time,Received Via,City,State,Zip code,Status,Filing on Behalf of Someone,Description
0,250635,Comcast Cable Internet Speeds,4/22/2015,3:53:50 PM,Internet,Abingdon,Maryland,21009,Closed,No,I have been contacting Comcast Internet Techni...
1,223441,Payment disappear - service got disconnected,4/8/2015,10:22:56 AM,Internet,Acworth,Georgia,30102,Closed,No,Back in January 2015 I made 2 payments: One fo...
2,242732,Speed and Service,4/18/2015,9:55:47 AM,Internet,Acworth,Georgia,30101,Closed,Yes,Our home is located at in Acworth Georgia 3010...
3,277946,Comcast Imposed a New Usage Cap of 300GB that ...,5/7/2015,11:59:35 AM,Internet,Acworth,Georgia,30101,Open,Yes,Comcast in the Atlanta area has just put into ...
4,307175,Comcast not working and no service to boot,5/26/2015,1:25:26 PM,Internet,Acworth,Georgia,30101,Solved,No,I have been a customer of Comcast of some sort...
...,...,...,...,...,...,...,...,...,...,...,...
2220,213550,Service Availability,4/2/2015,9:13:18 AM,Internet,Youngstown,Florida,32466,Closed,No,I am a deaf guy. I have asked ATT or Comcast t...
2221,318775,Comcast Monthly Billing for Returned Modem,6/2/2015,1:24:39 PM,Internet,Ypsilanti,Michigan,48197,Solved,No,We purchased our own modem and returned the Co...
2222,331188,complaint about comcast,6/9/2015,5:28:41 PM,Internet,Ypsilanti,Michigan,48197,Solved,No,i had an agreement with comcast agent 1 year f...
2223,360489,Extremely unsatisfied Comcast customer,6/23/2015,11:13:30 PM,Internet,Ypsilanti,Michigan,48197,Solved,No,A few months ago I was forced to finally call ...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Ticket #                     2225 non-null   object
 1   Customer Complaint           2225 non-null   object
 2   Date                         2225 non-null   object
 3   Time                         2225 non-null   object
 4   Received Via                 2225 non-null   object
 5   City                         2225 non-null   object
 6   State                        2225 non-null   object
 7   Zip code                     2225 non-null   int64 
 8   Status                       2225 non-null   object
 9   Filing on Behalf of Someone  2225 non-null   object
 10  Description                  2225 non-null   object
dtypes: int64(1), object(10)
memory usage: 191.3+ KB


### Complaint Categories

The column "customer Complaints" contains the different categories each complaint falls under. On further analysis, we find that there are 1842 categories. This probably is not the case, most customers file their issues under wrong categories by using LDA and LSA we can effectively classify these complaints into n number of topics of our choice.

In [25]:
df['Customer Complaint'].value_counts()

Comcast                                          83
Comcast Internet                                 18
Comcast Data Cap                                 17
comcast                                          13
Comcast Data Caps                                11
                                                 ..
Improper Billing and non resolution of issues     1
Deceptive trade                                   1
intermittent internet                             1
Internet Speed on Wireless Connection             1
Comcast, Ypsilanti MI Internet Speed              1
Name: Customer Complaint, Length: 1842, dtype: int64

# Data Pre-Processing

#### 1. Tokenization
#### 2. lemmatization
#### 3. stemming

In [26]:
def lemma_stem(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemma_stem(token))

    return result

### Regular Split Vs Stemmer Difference

In [27]:
print("Stemmed original document using a regular split(): ")
words = []
for word in df['Customer Complaint'].str.split(' '):
    words.append(word)
print(words[:5])

Stemmed original document using a regular split(): 
[['Comcast', 'Cable', 'Internet', 'Speeds'], ['Payment', 'disappear', '-', 'service', 'got', 'disconnected'], ['Speed', 'and', 'Service'], ['Comcast', 'Imposed', 'a', 'New', 'Usage', 'Cap', 'of', '300GB', 'that', 'punishes', 'streaming.'], ['Comcast', 'not', 'working', 'and', 'no', 'service', 'to', 'boot']]


In [28]:
stemmer = SnowballStemmer("english")
print(df['Customer Complaint'].fillna('').astype(str).map(preprocess))

0               [comcast, cabl, internet, speed]
1       [payment, disappear, servic, disconnect]
2                                [speed, servic]
3         [comcast, impos, usag, punish, stream]
4                  [comcast, work, servic, boot]
                          ...                   
2220                             [servic, avail]
2221       [comcast, month, bill, return, modem]
2222                        [complaint, comcast]
2223        [extrem, unsatisfi, comcast, custom]
2224       [comcast, ypsilanti, internet, speed]
Name: Customer Complaint, Length: 2225, dtype: object


In [29]:
processed_docs = []

for doc in df['Customer Complaint'].fillna('').astype(str):
    processed_docs.append(preprocess(doc))

# For LSA
df['Text (Clean)'] = df['Customer Complaint'].apply(lambda x: preprocess(x))

## Bag of Words ( BoW )


First we create a dictionary of the number of times each word occurs across the corpus by using the gensim package's dictionary function. This will be used as a parameter 'ID2' for our LDA model later.

In [30]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

# We now filter words that occur less than 5 times and those occurring more than half the time.
dictionary.filter_extremes(no_below=5, no_above=0.5)

0 cabl
1 comcast
2 internet
3 speed
4 disappear
5 disconnect
6 payment
7 servic
8 impos
9 punish
10 stream


### Gensim doc2bow

Each document in the corpus is assigned a BoW model by Doc2bow. An index of the number of times each word appears in a document is produced as a result.

In [31]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

## Latent Dirichlect Allocation (LDA)

In [32]:
lda_model =  gensim.models.LdaMulticore(bow_corpus,
                                        num_topics = 6,
                                        id2word = dictionary,
                                        passes = 10,
                                        workers = 2)

In [33]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.277*"internet" + 0.151*"bill" + 0.107*"speed" + 0.082*"servic" + 0.041*"throttl" + 0.038*"issu" + 0.036*"practic" + 0.034*"slow" + 0.024*"unfair" + 0.021*"connect"


Topic: 1 
Words: 0.143*"complaint" + 0.106*"servic" + 0.059*"bill" + 0.052*"bundl" + 0.040*"custom" + 0.039*"charg" + 0.038*"modem" + 0.037*"refus" + 0.026*"refund" + 0.024*"issu"


Topic: 2 
Words: 0.086*"issu" + 0.085*"cabl" + 0.061*"problem" + 0.053*"price" + 0.051*"lie" + 0.043*"monopoli" + 0.039*"servic" + 0.037*"instal" + 0.035*"increas" + 0.028*"fraud"


Topic: 3 
Words: 0.243*"servic" + 0.105*"xfiniti" + 0.077*"price" + 0.072*"custom" + 0.070*"charg" + 0.039*"internet" + 0.031*"poor" + 0.030*"provid" + 0.022*"cancel" + 0.019*"failur"


Topic: 4 
Words: 0.073*"servic" + 0.058*"advertis" + 0.049*"contract" + 0.044*"fals" + 0.043*"decept" + 0.042*"busi" + 0.040*"overcharg" + 0.036*"month" + 0.033*"switch" + 0.032*"block"


Topic: 5 
Words: 0.317*"data" + 0.120*"cap" + 0.068*"usag" + 0.043*"charg" + 

#### The model has produced five outputs. Using these five topics, it is possible to categorize all the words in the corpus. The LDA model, however, does not provide us with the topic names. we infer that from the results or we could use print_topics from pprint.

#### Manual Topic Classification Based on the above output

1. Concern
2. High-speed
3. Disappointing
4. Surcharge
5. Deceit
6. Bandwidth

#### Evaluation

There are two methods that could be leveraged to evaluate a LDA model, coherence and perplexity. Uncertainty is measured by perplexity, so the lower the perplexity, the more accurate the model. This is what we will be using today. As you can see the model performed well with a score of -4.58901


In [34]:
print('Perplexity Metric: ', lda_model.log_perplexity(bow_corpus))

Perplexity Metric:  -4.589018458625131


## Latent Semantic Analysis ( LSA )

In [35]:
# Calculate the coherence score when you have a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow_corpus, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} Topics: {}'.format(i, coherence_score))

Coherence score with 2 Topics: 0.4645335830295454
Coherence score with 3 Topics: 0.5198620542268798
Coherence score with 4 Topics: 0.5055562631148034
Coherence score with 5 Topics: 0.49671675404803917
Coherence score with 6 Topics: 0.5762340683961888
Coherence score with 7 Topics: 0.5198443217579267
Coherence score with 8 Topics: 0.5670752174275087
Coherence score with 9 Topics: 0.535671527047652
Coherence score with 10 Topics: 0.5509608793514204


#### When performing SVD, we will extract the topics with the highest coherence score. The strongest associations each topic has with 5 words are selected here.

In [39]:
# Extract 2 topics from the bag of words using SVD and the LsiModel
lsi = LsiModel(bow_corpus, num_topics=6, id2word=dictionary)

In [40]:
for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.677*"servic" + 0.645*"internet" + 0.197*"bill" + 0.164*"speed" + 0.122*"custom".
Words in 1: 0.675*"internet" + -0.607*"servic" + 0.252*"speed" + -0.239*"bill" + -0.150*"custom".
Words in 2: 0.870*"bill" + -0.322*"servic" + 0.228*"practic" + 0.188*"issu" + 0.153*"unfair".
Words in 3: -0.886*"data" + -0.376*"cap" + -0.169*"usag" + -0.108*"charg" + 0.106*"bill".
Words in 4: 0.896*"speed" + -0.321*"internet" + 0.158*"throttl" + 0.116*"charg" + 0.088*"advertis".
Words in 5: -0.939*"charg" + -0.136*"price" + 0.112*"speed" + -0.107*"modem" + 0.094*"data".


#### We can now see how the model scores each word based on the topic assigned to it. Documents can contain a number of topics, but some topics are closely related. This helps us determine the topic a document is associated with.

In [41]:
corpus_lsi = lsi[bow_corpus]
score1 = []
score2 = []
for doc in corpus_lsi:
    score1.append(doc)
    score2.append(doc)

# create data frame that shows scores assigned for both topics for each review
df_topic = pd.DataFrame()
df_topic['Text'] = df['Text (Clean)']
df_topic['Topic 0 score'] = score1
df_topic['Topic 1 score'] = score2
from operator import itemgetter
df_topic['Topic']= df_topic[['Topic 0 score', 'Topic 1 score']].apply(lambda x: x.max(), axis=1)
df_topic.head(1)

,Text,Topic 0 score,Topic 1 score,Topic
0,"[comcast, cabl, internet, speed]","[(0, 0.84603426514624), (1, 0.9635160138261774...","[(0, 0.84603426514624), (1, 0.9635160138261774...","[(0, 0.84603426514624), (1, 0.9635160138261774..."


The above is just an example with 2 topics, but it shows you the score related to each topic of a document allowing us to determine which topic best resembles the document.